In [1]:
from tqdm.notebook import tqdm

In [15]:
MAP_HEIGHT = 40
MAP_WIDTH = 40
WEIGHT_0 = 0.55
WEIGHTS = [WEIGHT_0, 1 - WEIGHT_0]
OPTIONS = [0, 1]
ITERATIONS = 5
# 0/▢ - floor, 1/▨ - wall, 2/▩ - map border, 3/◈ - treasure, 4/◎ - spawn point
CHARS = {
    0 : '\u25A2',
    1 : '\u25A8',
    2 : '\u25A9',
    3 : '\u25C8',
    4 : '\u25CE', 
}

STARVATION = 4
BIRTH = 3

In [3]:
def print_map(grid, file=None):
    print(file=file)
    for row in grid:
        row_str = ""
        for c in row:
            row_str += CHARS[c]
        print(row_str, file=file)

In [67]:
import random
from numpy import array, zeros_like, sum, mean
def initialize_grid():
    grid = []
    for _ in range(MAP_WIDTH):
        row = [random.choices(OPTIONS, WEIGHTS)[0] for _ in range(MAP_HEIGHT)]
        grid.append(row)
    return array(grid)

In [5]:
grid = initialize_grid()
print_map(grid)


▨▨▢▢▨▨▢▨▢▢▢▢▨▢▢▨▢▢▨▢▢▢▨▢▢▨▨▢▢▨▢▨▨▨▢▢▨▢▨▢
▢▨▢▨▨▢▢▨▢▨▨▨▢▨▢▨▢▨▨▢▨▢▢▢▢▨▨▢▢▨▢▨▢▢▨▢▨▢▨▨
▢▨▨▢▨▨▨▨▢▢▨▢▢▢▢▨▨▢▢▨▨▨▢▨▢▨▨▨▢▢▢▨▨▨▢▨▨▨▨▨
▨▨▢▢▢▨▨▢▨▢▨▨▨▢▢▢▢▢▢▢▢▢▢▢▨▢▨▢▨▨▢▢▨▨▢▨▨▨▢▢
▨▨▢▨▢▢▢▨▨▢▨▢▢▨▨▨▢▨▨▢▢▨▢▨▢▢▢▢▢▢▢▨▢▢▢▨▢▢▨▢
▢▨▨▨▢▢▢▨▢▨▢▢▢▨▨▢▢▢▨▢▢▨▨▢▢▨▨▨▨▨▨▨▢▨▨▢▨▨▨▢
▨▢▨▨▨▢▢▨▨▢▨▢▢▢▢▢▨▢▢▢▢▢▢▨▢▢▢▨▢▨▢▢▨▨▨▨▢▨▢▢
▨▨▢▨▢▢▢▢▢▨▢▢▨▨▨▢▢▢▢▢▢▢▢▢▨▨▢▢▢▨▨▨▢▨▨▢▨▨▨▨
▨▨▢▨▨▢▢▢▨▢▢▨▨▨▨▢▢▢▢▢▨▢▨▨▢▨▨▢▢▢▢▢▨▨▨▢▢▢▨▨
▢▨▢▢▨▢▢▨▨▨▨▨▨▨▨▢▨▨▢▢▨▢▢▨▨▨▢▢▢▢▨▢▨▨▨▨▢▢▢▢
▨▨▢▨▢▢▢▢▢▢▢▨▢▨▢▢▨▢▢▢▨▨▢▢▢▢▢▨▨▢▨▨▢▢▢▢▨▢▢▢
▨▢▢▢▢▢▢▨▢▨▢▢▨▨▨▨▨▢▢▢▨▨▨▨▢▨▢▢▨▨▢▢▨▨▨▢▢▨▢▢
▨▢▨▨▨▨▢▢▨▢▨▢▢▨▢▨▢▨▨▢▢▢▢▨▨▨▢▨▢▨▢▨▨▢▢▨▨▨▨▨
▢▨▢▢▢▨▢▢▢▨▨▢▢▨▨▢▢▢▢▨▢▢▢▨▢▨▨▨▢▨▢▨▢▢▨▨▢▢▢▢
▢▨▨▨▢▢▨▢▨▢▨▨▨▢▨▢▨▨▨▢▢▢▢▢▨▨▨▢▨▢▢▢▢▢▨▢▢▨▢▢
▢▢▢▨▨▨▢▢▢▢▢▢▢▨▨▨▢▨▨▢▢▨▢▢▨▨▢▨▢▨▢▨▢▨▢▨▢▢▢▢
▨▨▨▢▢▢▨▢▢▢▢▨▢▢▨▢▨▨▨▨▢▢▢▨▢▨▨▨▨▨▨▢▢▢▢▢▢▢▨▢
▢▢▨▢▨▢▢▨▨▢▢▢▢▨▢▨▨▢▢▢▢▨▨▢▢▢▨▢▨▨▢▢▢▨▢▢▢▢▢▨
▢▨▨▢▢▨▨▢▨▢▢▨▨▨▢▢▢▨▨▢▨▢▨▨▢▢▢▨▨▨▨▢▢▢▢▨▨▨▨▨
▢▢▢▢▨▢▢▢▨▨▢▢▢▢▢▨▢▨▢▨▢▢▢▨▨▨▨▢▨▢▢▨▨▢▨▢▢▢▢▢
▨▢▢▨▢▢▢▢▨▨▢▨▢▢▢▢▢▨▢▨▨▢▨▨▨▨▢▨▢▨▢▨▨▨▢▨▨▢▨▢
▨▨▨▢▨▢▨▢▨▨▨▢▢▢▢▢▢▢▨▢▨▢▨▨▨▢▢▢▢▨▢▨▨▨▢▢▢▨▨▨
▨▢▢▢▨▢▢▢▨▢▨▨▢▨▢▢▨▢▢▢▨▨▢▢▨▢▢▨▢▨▨▢▢▨▢▨▨▨▨▢
▢▢▨▨▢▢▢▨▨▨▨▨▨▨▨▨▢▢▨▨▢▢▢▨▨▨▢▨▢▨▨▨▨▨▢▨▨▢▨▢
▨▢▨▢▢▨▢▨▨▨▨▢▢▨▢

In [6]:
def count_alive_neighbors(board, x, y, n):
    """Sums the value of a 2D array's slice n-steps in every direction from P(x,y)"""
    try:
        value = board[y][x]
    except IndexError:
        return None
    board_width = len(board[0])
    board_height = len(board)
    board_slice = [row[max(x-n, 0):min(x+n+1, board_width)] for row in board[max(0, y-n):min(y+n+1, board_height)]]
    return sum(board_slice) - value

In [50]:
def calculate_new_state(old_value, alive_neighbors_1, alive_neighbors_2):
    if old_value:
        if alive_neighbors_1 <= STARVATION:
            return 0
        else:
            return 1
    else:
        if alive_neighbors_2 <= BIRTH:
            return 1
        else:
            return 0

def do_step(board):
    new_board = zeros_like(board)
    for y, row in enumerate(new_board):
        for x, _ in enumerate(row):
            old_element = board[y][x]
            alive_neighbors_1 = count_alive_neighbors(board, x, y, 1)
            alive_neighbors_2 = count_alive_neighbors(board, x, y, 2)
            new_board[y][x] = calculate_new_state(old_element, alive_neighbors_1, alive_neighbors_2)    
    return new_board

def add_border(board):
    for y, row in enumerate(board):
        for x, _ in enumerate(row):
            # I'd like the boundary of the map to stay as walls :D
            if x == 0 or x == MAP_WIDTH-1 or y == 0 or y == MAP_HEIGHT -1:
                board[y][x] = 2

### Trying to find the best values for BIRTH, STARVATION and ITERATIONS from between 0 and 8
- 0 is going to be wrong for iterations of course
- I'll be looking for results that offer the least variance and where walls cover up around half of the map

In [8]:
def get_coverage(grid):
    """Calculates how much of the real estate in the middle of the map is taken by walls in relation to 0.5,
       eg. 0 is 50% coverage, 1 is only walls, -1 is only floor tiles"""
    border = 2*MAP_WIDTH + 2*MAP_HEIGHT - 4
    full = MAP_WIDTH * MAP_HEIGHT - border
    walls = sum(grid) - border
    return (walls/full - 0.5) * 2

In [9]:
import statistics

MAP_WIDTH = 40
MAP_HEIGHT = 40

POSSIBILITIES = list(range(9))

random.seed(123)
SEEDS = [random.randrange(1_000_000) for _ in range(20)]

settings_cvgs = {}

with open("grids_settings_testing.txt", "w", encoding='utf8') as f:
    for death in tqdm(POSSIBILITIES):
        for birth in tqdm(POSSIBILITIES):
            for iterations in tqdm(POSSIBILITIES):
                if iterations < 1:
                    continue
                cvgs = []
                STARVATION = death
                BIRTH = birth
                ITERATIONS = iterations
                for seed in tqdm(SEEDS):                    
                    random.seed(seed)
                    grid = initialize_grid()
                    for _ in range(ITERATIONS):
                        grid = do_step(grid)
                    grid = add_border(grid)
                    print(f"SEED -{seed}-\n STARVATION IF BELOW: {STARVATION}, BIRTH AT {BIRTH}, {ITERATIONS} ITERATIONS", file=f)
                    cvgs.append(get_coverage(grid))
                    print_map(grid, f)
                info = {
                    "mean" : statistics.mean(cvgs),
                    "median" : statistics.median(cvgs),
                    "quantiles" : statistics.quantiles(cvgs),
                    "stdev" : statistics.stdev(cvgs),
                    "variance" : statistics.variance(cvgs)
                }
                settings_cvgs[f"{STARVATION}-{BIRTH}-{ITERATIONS}"] = info

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(settings_cvgs).T
df.to_excel("coverages.xlsx")

In [10]:
STARVATION = 4
BIRTH = 1
ITERATIONS = 3

with open(f"grids_{STARVATION}_{BIRTH}_{ITERATIONS}_testing.txt", "w", encoding='utf8') as f:
    for sd in list(range(50)):
        random.seed(sd)
        grid = initialize_grid()
        for _ in range(ITERATIONS):
            grid = do_step(grid)
        print(f"SEED {sd}", file=f)
        print_map(grid, f)

In [11]:
import json
with open("mockgrid.json", "w") as j:
    json.dump(grid.tolist(), j)

In [88]:
MAP_WIDTH = 80
MAP_HEIGHT = 80

STARVATION = 4
BIRTH = 1
ITERATIONS = 3

random.seed(1234)

grid_for_processing = initialize_grid()
for _ in range(ITERATIONS):
    grid_for_processing = do_step(grid_for_processing)
print_map(grid_for_processing)


▨▢▢▢▢▨▢▢▢▨▨▢▢▢▢▢▨▢▢▢▨▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▨▨▨▢
▨▢▢▢▢▨▢▢▨▨▨▨▢▢▢▢▨▢▢▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▨▨▨
▨▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▢▢▢▢▢▨▨▨▢▢▢▢▢▢▢▨▨▨▨
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▢▢▢▨▨▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▨▨▨▢▢▢▨▢▢▨▨▨▨▨▢▢▢▨▢▢▨▨▨▨▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▨▨▨▨
▢▢▢▢▢▢▢▨▢▢▢▢▢▢▨▨▢▢▢▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨▨▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▨
▢▢▢▨▢▢▨▨▨▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▢▨▨▨▨▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨
▢▢▢▨▢▢▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▨▨▢▢▢▨▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▢▢▨▨▨▢▢▢▢▨▨
▢▢▢▨▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨▢▨▨▨▨▨▨▢▢▢▢▢▢▨▢▢▢▢▢▨▢▢▢▢▢▢▢▨▨▢▢▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▢▢▨▨▨▨▨▢▢▨▨
▢▢▢▨▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢▢▨▢▢▢▢▢▨▢▢▢▢▢▢▢▢▨▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▢▢▨▨▢▢▢▢▢▨▨
▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▨▨
▢▢▢▢▢▢▢▨▢▨▨▨▨▨▨▨▨▢▢▢▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▨▢▢▢▨▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▨▨▨
▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▢▨▢▢▢▢▢▨▢▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢
▢▨▨▨▨▨▢▢▢▢▢▢▢▢▨▨▨▨▢▢▢▢▢▢▢▢▢

### Looking for a way to add treasure and spawn points
- I'll use a flood fill algoritm to find all accessible regions and at the same time find suitable areas for treasure placement:
    - places surrounded by walls from all but 1-2 sides make for good treasure hiding spots,
    - more open spaces would be good for enemies spawn points

In [89]:
# let's start by finding an empty spot somewhere in the middle of the map

def find_starting_spot_for_flood_fill(grid):
    x = random.randrange(int(0.3*MAP_WIDTH), int(0.7*MAP_WIDTH))
    y = random.randrange(int(0.3*MAP_WIDTH), int(0.7*MAP_WIDTH))
    while grid[y][x] != 0:
        x = random.randrange(int(0.3*MAP_WIDTH), int(0.7*MAP_WIDTH))
        y = random.randrange(int(0.3*MAP_WIDTH), int(0.7*MAP_WIDTH))
    return x, y

def flood_fill(x, y, grid, real_grid, treasure_locations, spawn_points):
    if grid[y][x] == 0:
        if count_alive_neighbors(real_grid, x, y, 1) > 4:
            treasure_locations.append((x, y))
        elif count_alive_neighbors(real_grid, x, y, 2) < 2:
            spawn_points.append((x, y))
        grid[y][x] = 99
        global filled_space
        filled_space += 1
        if x > 0:
            flood_fill(x-1, y, grid, real_grid, treasure_locations, spawn_points)
        if x < len(grid[0]) - 1:
            flood_fill(x+1, y, grid, real_grid, treasure_locations, spawn_points)
        if y > 0:
            flood_fill(x, y-1, grid, real_grid, treasure_locations, spawn_points)
        if y < len(grid) - 1:
            flood_fill(x, y+1, grid, real_grid, treasure_locations, spawn_points)

def fill_not_accessible_areas(grid, flood_filled_grid):
    for y, row in enumerate(grid):
        for x, _ in enumerate(row):
            if grid[y][x] == 0 and grid[y][x] == flood_filled_grid[y][x]:
                grid[y][x] = 1
                continue
    return grid

def filter_treasure_locations(treasure_locations):
    # I think a 70% chance for each one to be present on the map will be enough for now
    treasure_locations = [loc for loc in treasure_locations if random.choices([True, False], weights=[0.7, 0.3])[0]]
    return treasure_locations
    
def filter_spawn_points(spawn_points, grid):
    tmp_grid = array(grid, copy=True)
    add_spawn_points(tmp_grid, spawn_points)
    # I'd like to add all the ones that are separate from the other ones and remove them from the overall group
    # because of how they are initially selected, I can just use the count alive neighbors function
    # the "alone" spawn points will have 0 or 1 "alive neighbors" (a spawn point counts as 4)
    selected_spawn_points = [pt for pt in spawn_points if count_alive_neighbors(tmp_grid, *pt, 1) < 2]
    spawn_points = [pt for pt in spawn_points if not pt in selected_spawn_points]
    # the remaining spawn points create distinct groups, so I thought I'd use some simple clustering method
    # and then take the cluster's centroids as spawn points
    import scipy.cluster.hierarchy as hcluster
    clusters = hcluster.fclusterdata(spawn_points, 1.5, criterion="distance")
    spawn_points = [mean([spawn_points[i] for i, cl_no in enumerate(clusters) if cl_no == cl], axis=0) for cl in set(clusters)]
    # round coordinates and omit ones that land on a wall
    spawn_points = [(int(x), int(y)) for x, y in spawn_points if grid[int(y)][int(x)] == 0]
    selected_spawn_points.extend(spawn_points)
    return selected_spawn_points

def add_treasure_locations(grid, treasure_locations):
    for x, y in treasure_locations:
        grid[y][x] = 3
        
def add_spawn_points(grid, spawn_spots):
    for x, y in spawn_spots:
        grid[y][x] = 4
        
filled_space = 0
new_grid = array(grid_for_processing, copy=True)

potential_treasure_locations = []
potential_spawn_spots = []        
x, y = find_starting_spot_for_flood_fill(new_grid)
flood_fill(x, y, new_grid, grid_for_processing, potential_treasure_locations, potential_spawn_spots)                
grid_for_processing = fill_not_accessible_areas(grid_for_processing, new_grid)
print(len(potential_treasure_locations))
potential_treasure_locations = filter_treasure_locations(potential_treasure_locations)
print(len(potential_treasure_locations))
add_treasure_locations(grid_for_processing, potential_treasure_locations)
print(len(potential_spawn_spots))
potential_spawn_spots = filter_spawn_points(potential_spawn_spots, grid_for_processing)
print(len(potential_spawn_spots))
add_spawn_points(grid_for_processing, potential_spawn_spots)
add_border(grid_for_processing)

print(filled_space/(MAP_WIDTH*MAP_HEIGHT))
print_map(grid_for_processing)

48
27
351
45
0.560625

▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩▩
▩▢▢▢▢▨▢▢▨▨▨▨▢▢▢▢▨▢▢▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢◎▢▢▨▨▨▨▨▨▢▢◎▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▢▢▢▨▨▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▨▨▨▢▢▢▨▢▢▨▨▨▨▨▢▢▢▨▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▨▢▢▢▢▢▢▨▨▢▢▢▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▨▢▢▨▨▨▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▢▢▢◈▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▨▢▢▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▢▢◎▢▢▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▨▨▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▨▢▢▢▨▨▢▢▢▢▢▢▢▢▢▨▨▨◈▨▨▨▨▨▨▢▢▢▢▢▢▨▢▢◎▢▢▨▢▢▢▢▢▢▢▨▨▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▨▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢◈▨▨▨▨▨◈▢▢▢▢▢▢▢▢▨▢▢▢▢▢▨▢▢◎▢◎▢▢▢▨▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢▨▨▨▨▨▢▢▢▢▢▢▢▢◈▨▢▢▢▢▢▢▢▨▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▨▢▨▨▨▨▨▨▨▨▢▢▢▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨◈▢▢▢▢▢▢▢▢▢▢▨▢▢▢▨▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▢▨▢▢▢▢▢▨▨▨▨▨▨▨▨▢▢◎▢▢▢▢▨▨▨▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▩
▩▨▨▨▨

In [ ]:
import json
with open("mockgrid_big.json", "w") as j:
    json.dump(grid.tolist(), j)